In [1]:
import random
import re 

import pandas as pd

from tqdm.auto import tqdm

In [2]:
metadata = pd.read_json('/pio/scratch/1/recommender_systems/raw/Amazon/meta_Clothing_Shoes_and_Jewelry.json.gz', lines=True, compression='gzip')

In [46]:
ratings = pd.read_csv('/pio/scratch/1/i313924/data/train_data/slim_ratings.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'])
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [47]:
popular_items = ratings['asin'].unique()

In [28]:
suits = metadata[metadata['title'].str.contains('suit|tuxedo', regex=True, flags=re.IGNORECASE).fillna(False)]
suits = suits[suits.category.apply(lambda x: 'Men' in x)]

In [41]:
dresses = metadata[metadata['title'].str.contains('dress', regex=True, flags=re.IGNORECASE).fillna(False)]
dresses = dresses[dresses.category.apply(lambda x: 'Women' in x)]
dresses = dresses.loc[dresses['asin'].isin(popular_items)]

In [48]:
suits = suits.loc[suits['asin'].isin(popular_items)]
dresses = dresses.loc[dresses['asin'].isin(popular_items)]

In [50]:
len(suits)

5119

In [51]:
len(dresses)

9592

In [56]:
test_set = []
for _ in tqdm(range(10000)):
    positive = 'Women'
    negative = 'Men'
    if random.random() < 0.5:
        positive, negative = negative, positive
    if positive == 'Women':
        positive_items = dresses.sample(2)['asin'].values
        negative_item = suits.sample(1)['asin'].values
    else:
        positive_items = suits.sample(2)['asin'].values
        negative_item = dresses.sample(1)['asin'].values

    line = {"A": positive_items[0],
            "B": negative_item[0],
            "X": positive_items[1],
            "category_AX": positive,
            "category_B": negative
           }
    test_set.append(line)
with open('/pio/scratch/1/i313924/data/test_data/Suits_Dresses_ABX', 'w') as file:    
    pd.DataFrame(test_set).to_json(file, 'records', lines = True)

  0%|          | 0/10000 [00:00<?, ?it/s]